In [21]:
import sys
sys.path[0]+='/..'
from utilities import *
import torch
import torch_geometric
import gurobipy as gp
from uc_class import UC
import copy
from torch_geometric.data import HeteroData


In [3]:
class BipartiteGraphConvolution(torch_geometric.nn.MessagePassing):
    """
    The bipartite graph convolution is already provided by pytorch geometric and we merely need
    to provide the exact form of the messages being passed.
    """

    def __init__(self):
        super().__init__("add")
        emb_size = 64
            
        self.feature_module_left = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size)
        )
        self.feature_module_edge = torch.nn.Sequential(
            torch.nn.Linear(1, emb_size, bias=False)
        )
        self.feature_module_right = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size, bias=False)
        )
        self.feature_module_final = torch.nn.Sequential(
            torch.nn.LayerNorm(emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

        self.post_conv_module = torch.nn.Sequential(torch.nn.LayerNorm(emb_size))

        # output_layers
        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(2 * emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

    def forward(self, left_features, edge_indices, edge_features, right_features):
        """
        This method sends the messages, computed in the message method.
        """
        output = self.propagate(
            edge_indices,
            size=(left_features.shape[0], right_features.shape[0]),
            node_features=(left_features, right_features),
            edge_features=edge_features,
        )

        return self.output_module(
            torch.cat([self.post_conv_module(output), right_features], dim=-1)
        )


    def message(self, node_features_i, node_features_j, edge_features):
        #node_features_i,the node to be aggregated
        #node_features_j,the neighbors of the node i

        output = self.feature_module_final(
            self.feature_module_left(node_features_i)
            + self.feature_module_edge(edge_features)
            + self.feature_module_right(node_features_j)
        )

        return output
    

In [58]:
class GNNPolicy(torch.nn.Module):
    def __init__(self, in_channels=64, hidden_channels=64, out_channels=64):
        # num_relations=
        super(GNNPolicy, self).__init__()
        self.var_node = ['u','p','s']
        self.con_node = ['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.node_types=['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.relations = ['v2c','c2v']
        self.init_sizes=[6,6,6,4,4,4,4,4,4,4]
        edge_feats=1

        self.num_relations=len(self.var_node)*len(self.con_node)
        
        self.vc_idx={}

        self.node_types = ['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']    

        self.convc1 = torch.nn.ModuleList()
        self.concv1 = torch.nn.ModuleList()
        self.convc2 = torch.nn.ModuleList()
        self.concv2 = torch.nn.ModuleList()

        for i in self.var_node:
            for j in self.con_node:
                self.convc1.append(BipartiteGraphConvolution())
                self.concv1.append(BipartiteGraphConvolution())
                self.convc2.append(BipartiteGraphConvolution())
                self.concv2.append(BipartiteGraphConvolution())



        self.node_embedding = torch.nn.ModuleList()
        for i in range(len(self.node_types)):
            lin = torch.nn.Sequential(
                torch.nn.LayerNorm(self.init_sizes[i]),
                torch.nn.Linear(self.init_sizes[i], hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels,hidden_channels),
                torch.nn.ReLU()
            )
            self.node_embedding.append(lin)

        self.edge_embedding = torch.nn.ModuleList()
        for i in range(self.num_relations):
            lin = torch.nn.LayerNorm(edge_feats)
            self.edge_embedding.append(lin)
        
        self.linv1 = torch.nn.Linear(hidden_channels*2, hidden_channels)
        self.linc1 = torch.nn.Linear(hidden_channels*2, hidden_channels)

        self.linv2 = torch.nn.Linear(out_channels*2, out_channels)
        self.linc2 = torch.nn.Linear(out_channels*2, out_channels)


        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(out_channels, int(out_channels/2)),
            torch.nn.ReLU(),
            torch.nn.Linear(int(out_channels/2), 1, bias=False)
        )

    def trans_dimensions(self, g):
        data = copy.deepcopy(g)
        for node_type, lin in zip(self.node_types, self.node_embedding):
            data[node_type].x = lin(data[node_type].x)
        cnt=0
        for v in self.var_node:
            for c in self.con_node:
                if 'edge_attr' in data[v,'v2c',c]:
                    data[v,'v2c',c].edge_attr = self.edge_embedding[cnt](data[v,'v2c',c].edge_attr)
                    data[c,'v2c',v].edge_attr = data[v,'v2c',c].edge_attr
                    self.vc_idx[(v,c)]=cnt
                else:
                    self.vc_idx[(v,c)]=-1
                cnt+=1
        return data

    def forward(self, data):
        data = self.trans_dimensions(data)
        for c in self.con_node:
            x = torch.zeros_like(data[c].x)
            # print(x.shape)
            for v in self.var_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.convc1[self.vc_idx[(v,c)]](data[v].x, data[v,'v2c',c].edge_index, data[v,'v2c',c].edge_attr,data[c].x)
            data[c].x = self.linc1(torch.cat([data[c].x, x], dim=-1))



        for v in self.var_node:
            x = torch.zeros_like(data[v].x)
            for c in self.con_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.concv1[self.vc_idx[(v,c)]](data[c].x, data[c,'c2v',v].edge_index, data[c,'c2v',v].edge_attr,data[v].x)
            data[v].x = self.linv1(torch.cat([data[v].x, x], dim=-1))

        for c in self.con_node:
            x = torch.zeros_like(data[c].x)
            for v in self.var_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.convc2[self.vc_idx[(v,c)]](data[v].x, data[v,'v2c',c].edge_index, data[v,'v2c',c].edge_attr,data[c].x)
            data[c].x = self.linc2(torch.cat([data[c].x, x], dim=-1))

        for v in self.var_node:
            x = torch.zeros_like(data[v].x)
            for c in self.con_node:
                if self.vc_idx[(v,c)]!=-1:
                    x += self.concv2[self.vc_idx[(v,c)]](data[c].x, data[c,'c2v',v].edge_index, data[c,'c2v',v].edge_attr,data[v].x)
            data[v].x = self.linv2(torch.cat([data[v].x, x], dim=-1))

        

        x = self.output_module(data['u'].x).sigmoid()
        
        return x

In [60]:
def get_graph(m):
    mvars = m.getVars()
    constrs = m.getConstrs()
    b_vars = torch.tensor([1 if 'u'in v.Varname else 0 for v in mvars])
    coo = m.getA().tocoo()
    # values = coo.data
    # indices = np.vstack((coo.row, coo.col))
    v0 = torch.tensor(m.getAttr("Obj", mvars)).reshape(-1,1)
    v1 = torch.tensor(coo.getnnz(axis=0)).reshape(-1,1)
    v2 = torch.tensor(coo.sum(axis=0)).reshape(-1,1)/v1
    v3 = torch.tensor(coo.max(axis=0).toarray()).reshape(-1,1)
    v4 = torch.tensor(coo.min(axis=0).toarray()).reshape(-1,1)
    v5 = b_vars.clone().reshape(-1,1)
    v_nodes = torch.cat((v0,v1,v2,v3,v4,v5),1).float()
    c0 = torch.tensor(coo.getnnz(axis=1)).reshape(-1,1)
    c1 = torch.tensor(coo.sum(axis=1)).reshape(-1,1)/c0
    c2 = torch.tensor(m.getAttr("RHS", constrs)).reshape(-1,1)
    c3 = torch.tensor([2 if i=='<' else 1 for i in m.getAttr("Sense", constrs)]).reshape(-1,1)
    c_nodes = torch.cat((c0,c1,c2,c3),1).float()

    cons=['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']

    graph = HeteroData()

    n=len(mvars)
    T=24
    N=int(sum(b_vars)/T)


    if N*T == n/3:
        
        graph['u'].x = v_nodes[:N*T]
        graph['p'].x = v_nodes[N*T:2*N*T]
        graph['s'].x = v_nodes[2*N*T:3*N*T]

        r = 0
        idx=0
        for l, c in enumerate(constrs):
            if cons[idx] in c.ConstrName and l != len(constrs) - 1:
                continue
            if l == len(constrs) - 1:
                l+=1
            graph[cons[idx]].x = c_nodes[r:l]


            uindice = []
            pindice = []
            sindice = []
            uvalues = []
            pvalues = []
            svalues = []

            for constr, variable, value in zip(coo.row, coo.col, coo.data):
                if r <= constr < l:
                    if variable < N*T:
                        uindice.append((variable, constr-r))
                        uvalues.append(value)
                    elif N*T <= variable < 2*N*T:
                        pindice.append((variable-N*T, constr-r))
                        pvalues.append(value)
                    elif variable >= 2*N*T:
                        sindice.append((variable-2*N*T, constr-r))
                        svalues.append(value)

            uindice = np.array(uindice).T
            pindice = np.array(pindice).T
            sindice = np.array(sindice).T
            uvalues = torch.tensor(uvalues, dtype=torch.float32).reshape(-1,1)
            pvalues = torch.tensor(pvalues, dtype=torch.float32).reshape(-1,1)
            svalues = torch.tensor(svalues, dtype=torch.float32).reshape(-1,1)

            if uindice.size:
                graph['u','v2c',cons[idx]].edge_index = torch.from_numpy(uindice)
                graph['u','v2c',cons[idx]].edge_attr = uvalues
                graph[cons[idx],'c2v','u'].edge_index = torch.from_numpy(uindice[[1,0]])
                graph[cons[idx],'c2v','u'].edge_attr = uvalues
            if pindice.size:
                graph['p','v2c',cons[idx]].edge_index = torch.from_numpy(pindice)
                graph['p','v2c',cons[idx]].edge_attr = pvalues
                graph[cons[idx],'c2v','p'].edge_index = torch.from_numpy(pindice[[1,0]])
                graph[cons[idx],'c2v','p'].edge_attr = pvalues
            if sindice.size:                
                graph['s','v2c',cons[idx]].edge_index = torch.from_numpy(sindice)
                graph['s','v2c',cons[idx]].edge_attr = svalues
                graph[cons[idx],'c2v','s'].edge_index = torch.from_numpy(sindice[[1,0]])
                graph[cons[idx],'c2v','s'].edge_attr = svalues
            r = l
            idx += 1
        graph
        return graph


In [61]:
file = '../UC_AF/5_std.mod'
uc = UC(file)
m=uc.get_1bin_model()
# print(m)
graph = get_graph(m)
# print(graph)

In [62]:
model = GNNPolicy()
model(graph)

tensor([[0.4915],
        [0.4939],
        [0.4929],
        [0.4950],
        [0.4931],
        [0.4954],
        [0.4906],
        [0.4957],
        [0.4915],
        [0.4933]], grad_fn=<SigmoidBackward0>)

In [15]:
# var_node = ['u','p','s']
# con_node = ['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']

# my_dict={}
# cnt=0
# for i in var_node:
#     for j in con_node:
#         my_dict[(i,j)]=cnt
#         my_dict[(j,i)]=cnt
#         cnt+=1
# print(my_dict[('u','Power_balance_constrains')])
# print(my_dict[('Power_balance_constrains','Power_balance_constrains')])

2


KeyError: ('Power_balance_constrains', 'Power_balance_constrains')